In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import pandas as pd

In [ ]:
PATH=r'~\ITPC\data'

def itpc(frequency=2.5,sfreq=512,n_cycles=7,start_index=102,end_index=2100,path=PATH): # set target frequency, sampling rate, number of cycles for the wavelet, start and end timepoint for averaging, directory path
    itpc_per=[]
    itpc_aper=[]
    itpc_visual=[]
    # Calculate the complex wavelet coefficients for the frequency of interest
    wavelet_coefficients = mne.time_frequency.morlet(sfreq, [frequency], n_cycles=n_cycles)[0]
    for i in os.listdir(path):
        # Load the .mat file
        data = scipy.io.loadmat(path+'\\'+i)
        # Access the 'PreProcData_Auditory_Periodic_Encoding' field (conditions)
        if 'Periodic' in i:
            data = data['PreProcData_Auditory_Periodic_Encoding']
        elif 'Aperiodic' in i:
            data = data['PreProcData_Auditory_Aperiodic_Encoding']
        elif 'Visual' in i:
            data = data['PreProcData_Visual_Encoding']
        itpc_values = []
        # Initialize an array to store the phase values for each channel
        phase_values_channels = []
        # Loop over each channel
        for channel in range(64): # the number of channels is fixed, modify if needed
            # Initialize an array to store the phase values for each trial
            phase_values_trials = []
            # Loop over each trial
            for trial in range(data['trial'][0][0][0].shape[0]):
                # Calculate the phase values for trial
                phase_values = np.angle(np.convolve(data['trial'][0][0][0][trial][channel], wavelet_coefficients, mode='same'))
                phase_values_trials.append(phase_values)
            itpc_channel = np.abs(np.mean(np.exp(1j * np.array(phase_values_trials)), axis=0))
            itpc_values.append(itpc_channel)
        # itpc_*condition* is a list containing the itpc_values list for each subject, for condition
        if 'Periodic' in i:
            itpc_per.append(itpc_values)
        elif 'Aperiodic' in i:
            itpc_aper.append(itpc_values)
        elif 'Visual' in i:
            itpc_visual.append(itpc_values)
    # itpc_*condition* is expected to be a list containing # subjects lists, each containing 64 (channels) (4148,) arrays with itpc values
    print("For the Periodic condition:\nNumber of subjects: ",len(itpc_per),"\nNumber of channels: ", len(itpc_per[0]),"\nNumber of timepoints:", len(itpc_per[0][0]))
    
    # Time averaging
    mean_itpc_per = []
    mean_itpc_aper = []
    mean_itpc_visual = []

    # Iterate over each subject
    for subject in range(len(itpc_per)):
        # Initialize lists to store the mean ITPC values for each condition
        mean_per_subject = []
        mean_aper_subject = []
        mean_visual_subject = []

        # Iterate over each channel
        for channel in range(len(itpc_per[subject])):
            # Calculate the mean ITPC for the time window for the 'Periodic' condition
            mean_per_channel = np.mean(itpc_per[subject][channel][start_index:end_index])
            mean_per_subject.append(mean_per_channel)

            # Calculate the mean ITPC for the time window for the 'Aperiodic' condition
            mean_aper_channel = np.mean(itpc_aper[subject][channel][start_index:end_index])
            mean_aper_subject.append(mean_aper_channel)

            # Calculate the mean ITPC for the time window for the 'Visual' condition
            mean_visual_channel = np.mean(itpc_visual[subject][channel][start_index:end_index])
            mean_visual_subject.append(mean_visual_channel)

        # Append the mean ITPC values for the subject to the corresponding lists
        mean_itpc_per.append(mean_per_subject)
        mean_itpc_aper.append(mean_aper_subject)
        mean_itpc_visual.append(mean_visual_subject)

    # Print the dimensions of the resulting mean ITPC arrays
    print("Dimensions of mean ITPC arrays:")
    print("Periodic: ", np.array(mean_itpc_per).shape)
    print("Aperiodic: ", np.array(mean_itpc_aper).shape)
    print("Visual: ", np.array(mean_itpc_visual).shape)
    
    # Convert the mean ITPC arrays to a DataFrame
    df_per = pd.DataFrame(mean_itpc_per, columns=[f'Channel{i+1}' for i in range(64)])
    df_aper = pd.DataFrame(mean_itpc_aper, columns=[f'Channel{i+1}' for i in range(64)])
    df_visual = pd.DataFrame(mean_itpc_visual, columns=[f'Channel{i+1}' for i in range(64)])
    
    path_to_save=r'~\ITPC\final\freq_'+str(frequency)+'\num_cycles_'+str(n_cycles)
    # Save the DataFrame for 'Periodic' condition
    output_per_path = path_to_save+'\periodic.csv'
    df_per.to_csv(output_per_path, index=False)

    # Save the DataFrame for 'Aperiodic' condition
    output_aper_path = path_to_save+'\aperiodic.csv'
    df_aper.to_csv(output_aper_path, index=False)

    # Save the DataFrame for 'Visual' condition
    output_visual_path = path_to_save+'\visual.csv'
    df_visual.to_csv(output_visual_path, index=False)

In [ ]:
# ROI grouping

# Define the ROI channel indices -- predefined
roi_indices = {
    'antleftch': list(range(3)) + list(range(4, 6)),
    'antrightch': list(range(33, 36)) + list(range(39, 41)),
    'antmidch': [3, 32] + list(range(36, 39)),
    'centleftch': list(range(8, 10)) + list(range(12, 14)) + list(range(16, 18)),
    'centrightch': list(range(43, 45)) + list(range(49, 51)) + list(range(53, 55)),
    'centmidch': list(range(10, 12)) + [18, 31] + list(range(45, 49)) + [55],
    'postleftch': list(range(20, 23)) + list(range(24, 26)),
    'postrightch': list(range(57, 60)) + list(range(61, 63)),
    'postmidch': [19] + list(range(28, 31)) + [56]
}

# Define the path to the directory containing the CSV files
directory_path = r'~\ITPC\final'

# Iterate over the subdirectories
for freq_directory in ['freq_2.5', 'freq_3.5']:
    freq_directory_path = os.path.join(directory_path, freq_directory)

    # Iterate over the num_cycles subdirectories
    for num_cycles_directory in ['num_cycles_7', 'num_cycles_9', 'num_cycles_11']:
        num_cycles_directory_path = os.path.join(freq_directory_path, num_cycles_directory)

        # Iterate over the CSV files
        for condition in ['periodic', 'aperiodic', 'visual']:
            csv_file_path = os.path.join(num_cycles_directory_path, f'{condition}.csv')

            # Load the CSV file as a DataFrame
            df = pd.read_csv(csv_file_path)

            # Create a new DataFrame to store the ROI averages
            roi_df = pd.DataFrame(index=df.index)

            # Iterate over the ROIs and calculate the average for each subject
            for roi, channels in roi_indices.items():
                roi_average = df.iloc[:, channels].mean(axis=1)
                roi_df[roi] = roi_average

            # Save the ROI DataFrame as a new CSV file
            output_path = os.path.join(r'~\ITPC\itpc_roi', f'{freq_directory}hz_{num_cycles_directory}_{condition}.csv')
            roi_df.to_csv(output_path, index=False)

            print(f"ROI averages for '{condition}' condition saved to CSV successfully.")

print("All ROI averages saved successfully.")